In [3]:
# !pip install ratsnlp

In [4]:
# !pip install pytorch-lightning==1.7.0

In [1]:
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments

2023-06-01 15:29:25.211360: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-01 15:29:25.257054: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 15:29:25.870850: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [158]:
args = GenerationTrainArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_corpus_name="nsmc",
    downstream_corpus_root_dir = "./Untitled Folder",
    downstream_model_dir="./Untitled Folder",
    max_seq_length=32,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=256,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [3]:
from ratsnlp import nlpbook

nlpbook.set_seed(args)
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='./Untitled Folder', downstream_model_dir='./Untitled Folder', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=256, batch_size=1, cpu_workers=16, fp16=False, tpu_cores=0)


set seed: 7


In [4]:
# from Korpora import Korpora
# Korpora.fetch(
#     corpus_name=args.downstream_corpus_name,
#     root_dir=args.downstream_corpus_root_dir,
#     force_download=args.force_download,
# )

In [5]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [35]:
from konlpy.tag import Okt
okt = Okt()

In [37]:
okt.morphs(df["value"][0])

['반려동물', '의', '종류', '에', '대해', '서', '알', '고', '계신가요', '?']

In [44]:
df.shape[0]

176827

In [49]:
dic = {}
key = 2
for i in range(len(df["value"])):
    for j in okt.morphs(df["value"][i]):
        if j in dic.keys():
            continue
        dic[j] = key
        key += 1
        if key%1000==0:
            print(key)
        
for m in range(len(df["label"])):
    for n in okt.morphs(df["label"][m]):
        if n in dic.keys():
            continue
        dic[j] = key
        key += 1
        if key%1000==0:
            print(key)
            
            

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000


In [140]:
dic["unk"] = 0
dic["pad"] = 1

In [141]:
reverse = dict(map(reversed,dic.items()))

In [6]:
# from ratsnlp.nlpbook.generation import NsmcCorpus, GenerationDataset
# corpus = NsmcCorpus()
# train_dataset = GenerationDataset(
#     args=args,
#     corpus=corpus,
#     tokenizer=tokenizer,
#     mode="train",
# )

In [118]:
df2 = pd.read_csv("./input_df.csv")
df2

,value,label
0,반려동물의 종류에 대해서 알고 계신가요?,네.
1,네.,알고 있습니다.
2,알고 있습니다.,보통 요즘 현대인들이 많이 키우는 반려동물은 개 고양이 위주로 많이 키우는데 그 외...
3,보통 요즘 현대인들이 많이 키우는 반려동물은 개 고양이 위주로 많이 키우는데 그 외...,저도 기회가 된다면 나중에는 고슴도치 같은 걸 키워보고 싶기도 하고 제 주위에 뱀을...
4,저도 기회가 된다면 나중에는 고슴도치 같은 걸 키워보고 싶기도 하고 제 주위에 뱀을...,키우는 반려동물이 있으신가요?
...,...,...
176822,아무래도 영상학과고 서로 협업해야 하는 과제가 많기 때문에 그런 경우가 있었는데요.,보통 선배들과 후배들이 친해지는 경우가 꽤 있고 그런 경우에는 선배가 후배에게 그 ...
176823,보통 선배들과 후배들이 친해지는 경우가 꽤 있고 그런 경우에는 선배가 후배에게 그 ...,아니면 과제가 많다 적다 아니면 시험은 이런 식으로 내신다 이런 식의 선배들이 도움...
176824,아니면 과제가 많다 적다 아니면 시험은 이런 식으로 내신다 이런 식의 선배들이 도움...,후배들은 선배가 취업 활동을 진행할 때 그에 다른 도움을 주거나 그~ 그런 식으로 ...
176825,후배들은 선배가 취업 활동을 진행할 때 그에 다른 도움을 주거나 그~ 그런 식으로 ...,대학교에는 다양한 장학금 제도가 진행되고 있다고 들었는데 장학금은 꼭 학점이 높은 ...


In [122]:
[dic[i] for i in okt.morphs(df2["value"][0])]

[2, 3, 4, 5, 6, 7, 8, 9, 10, 89180]

In [144]:
len(df["token_value_num"][0])

647

In [191]:
## Custom Dataset

import os
from ratsnlp.nlpbook.generation.corpus import GenerationExample
from torch.utils.data import Dataset, DataLoader

class NewsCorpus(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        input_sentence = self.df["value"][idx]
        label_sentence = self.df["label"][idx]
        
#         x = GenerationExample(text = input_sentence)
        token_ids = [dic[i] for i in okt.morphs(input_sentence)]
        x = torch.ones(647, dtype=torch.long)
        for i in range(len(token_ids)):
            x[i] = token_ids[i]
#         y = GenerationExample(text = label_sentence)
        label_ids = [dic[i] for i in okt.morphs(label_sentence)]
        y = torch.ones(647, dtype=torch.long)
        for i in range(len(label_ids)):
            y[i] = label_ids[i]
        
        return x,y

In [192]:
trainset = NewsCorpus(df2)
trainloader = DataLoader(trainset)

In [193]:
x,y = next(iter(trainloader))
print(x.shape, y.shape)
print(x.dtype, y.dtype)

torch.Size([1, 647]) torch.Size([1, 647])
torch.int64 torch.int64


In [70]:
import pandas as pd
df = pd.read_csv("./example_df2.csv")

In [148]:
# from ratsnlp.nlpbook.generation import GenerationDataset
# corpus = NewsCorpus(df)

# train_dataset = GenerationDataset(
#     args=args,
#     corpus=corpus,
#     tokenizer=tokenizer,
#     mode="train",
    
# )

In [194]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    trainset,
    batch_size=args.batch_size,
#     sampler=RandomSampler(trainset, replacement=False),
#     collate_fn=nlpbook.data_collator,
#     drop_last=False,
#     num_workers=args.cpu_workers,
)
# from torch.utils.data import SequentialSampler
# val_dataset = GenerationDataset(
#     args=args,
#     corpus=corpus,
#     tokenizer=tokenizer,
#     mode="test",
# )
# val_dataloader = DataLoader(
#     val_dataset,
#     batch_size=args.batch_size,
#     sampler=SequentialSampler(val_dataset),
#     collate_fn=nlpbook.data_collator,
#     drop_last=False,
#     num_workers=args.cpu_workers,
# )

In [195]:
x,y = next(iter(train_dataloader))
print(x.shape, y.shape)
print(x.dtype, y.dtype)

torch.Size([32, 647]) torch.Size([32, 647])
torch.int64 torch.int64


In [165]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    args.pretrained_model_name
)
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [175]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset
import torchaudio
from torch.utils.data import DataLoader
import os
import numpy as np
import torch.nn.functional as F

In [198]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.gru1 = nn.GRU(input_size, hidden_size, batch_first=True)
        self.gru2 = nn.GRU(hidden_size, hidden_size, batch_first=True)  # Additional GRU layer
        self.gru3 = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.gru4 = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
#         self.init = nn.init.xavier_normal_(tensor, gain=1.0)
#         self.softmax = nn.Softmax()
        
    def forward(self, x):
        h0 = nn.init.xavier_uniform_(torch.randn(1, x.size(0), self.hidden_size).to(x.device), gain=1.0)
        #c0 = torch.randn(1, x.size(0), self.hidden_size).to(x.device)
        hidden_state0 = h0.cpu().detach().numpy()
        output, hidden1 = self.gru1(x, (h0))
        hidden_state1 = hidden1.cpu().detach().numpy()
        output, hidden2 = self.gru2(self.relu(output), hidden1) # Pass output through the second LSTM layer
        hidden_state2 = hidden2.cpu().detach().numpy()
        output, hidden3 = self.gru3(self.relu(output), hidden2) # Pass output through the second LSTM layer
        hidden_state3 = hidden3.cpu().detach().numpy()
        output, hidden4 = self.gru4(self.relu(output), hidden3) # Pass output through the second LSTM layer
        hidden_state4 = hidden4.cpu().detach().numpy()
        output = self.fc(self.relu(output[:, -1, :]))
        return output, [hidden_state0, hidden_state1, hidden_state]

In [199]:
# model.lm_head = nn.Linear(768,647)
# model.transformer.wte = nn.Embedding(647, 768)
# model.transformer.wpe = nn.Embedding(647, 768)
# model

In [200]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GRUModel(x.shape[1], 512, y.shape[1]).to(device)
criterion = nn.MSELoss(reduction="mean")
optimizer = optim.SparseAdam(model.parameters(), lr=.01)
scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [197]:
from time import time
timelist = []

model.train()

num_epochs = 256

for epoch in range(num_epochs):
    total_loss = 0.0
    total_correct = 0
    start = time()
    if epoch%32 == 0:
        torch.save(model, "./Untitled Folder/Text2Text.pt")

    for i, (x,y) in enumerate(train_dataloader):
        x,y = x.to(device), y.to(device)
        
        outputs = model(x)
        loss = criterion(outputs, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        text = text.view(-1, 1)
        total_correct += (predicted == text).sum().item()

    scheduler.step()
    epoch_loss = total_loss / (batch_size * total_step)
    epoch_acc = total_correct / (batch_size * total_step)
    end = time()
    timelist.append(end-start)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}, Time Duration : {end - start:.4f}:')


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# from ratsnlp.nlpbook.generation import GenerationTask
# task = GenerationTask(model, args)
# trainer = nlpbook.get_trainer(args)

In [179]:
# trainer.fit(
#     task,
#     train_dataloaders=train_dataloader,
# #     val_dataloaders=val_dataloader,
# )

In [46]:
import
from ratsnlp.nlpbook.generation import GenerationDeployArguments
args = GenerationDeployArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_model_dir="./Untitled Folder",
)

In [47]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel
if args.downstream_model_checkpoint_fpath is None:
    model = GPT2LMHeadModel.from_pretrained(
        args.pretrained_model_name,
    )
else: 
    pretrained_model_config = GPT2Config.from_pretrained(
        args.pretrained_model_name,
    )
    model = GPT2LMHeadModel(pretrained_model_config)
    fine_tuned_model_ckpt = torch.load(
        # model,
        args.downstream_model_checkpoint_fpath,
        map_location=torch.device("cpu"),
    )
    model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

In [48]:
input_ids = tokenizer.encode("밥은 먹었어?", return_tensors="pt")

In [50]:
## https://ratsgo.github.io/nlpbook/docs/generation/inference1/

with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty = 1.5,
        no_repeat_ngram_size=3,
        top_k=50, 
        temperature=0.9, ## 1보다 작게 하면 정확한 문장을, 1보다 크게 하면 다양한 문장 생성
        top_p = 0.92,
      )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

In [ ]:
## https://ratsgo.github.io/nlpbook/docs/generation/inference2/#%EC%BD%94%EB%93%9C6-%EC%9D%B8%ED%8D%BC%EB%9F%B0%EC%8A%A4

def inference_fn(
        prompt,
        min_length=10,
        max_length=20,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.0,
        no_repeat_ngram_size=0,
        temperature=1.0,
):
    try:
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        with torch.no_grad():
            generated_ids = model.generate(
                input_ids,
                do_sample=True,
                top_p=float(top_p),
                top_k=int(top_k),
                min_length=int(min_length),
                max_length=int(max_length),
                repetition_penalty=float(repetition_penalty),
                no_repeat_ngram_size=int(no_repeat_ngram_size),
                temperature=float(temperature),
           )
        generated_sentence = tokenizer.decode([el.item() for el in generated_ids[0]])
    except:
        generated_sentence = """처리 중 오류가 발생했습니다. <br>
            변수의 입력 범위를 확인하세요. <br><br> 
            min_length: 1 이상의 정수 <br>
            max_length: 1 이상의 정수 <br>
            top-p: 0 이상 1 이하의 실수 <br>
            top-k: 1 이상의 정수 <br>
            repetition_penalty: 1 이상의 실수 <br>
            no_repeat_ngram_size: 1 이상의 정수 <br>
            temperature: 0 이상의 실수
            """
    return {
        'result': generated_sentence,
    }

In [ ]:
from ratsnlp.nlpbook.qa import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()